In [48]:
from torch.nn.utils.rnn import pad_sequence
import torch
fake_data = [[[(1, 2), (3, 4), (5, 6), (7, 8)]
, [(1, 2), (3, 4), (7, 8)]
, [(1, 2), (7, 8)]
, [(1, 2), (3, 4), (5, 6), (7, 8)],], 
[[(1, 2)]
, 0
, [(1, 2), (7, 8)]
, [(1, 2), (3, 4), (5, 6), (7, 8)],], 
[[(1, 2), (3, 4), (5, 6), (7, 8)]
, [(1, 2), (3, 4), (7, 8)]
, [(1, 2), (3, 4), (5, 6), (7, 8)],], 
[[(1, 2), (3, 4), (5, 6), (7, 8)]
, [(1, 2), (3, 4)],], 
]

In [49]:
import itertools
from itertools import islice
bins_per_trial = [len(trial) for trial in fake_data]
unfold_spikes = list(itertools.chain.from_iterable(fake_data))
new_unfold_spikes = [x if x != 0 else torch.empty(0, 2) for x in unfold_spikes]
print(new_unfold_spikes)

[[(1, 2), (3, 4), (5, 6), (7, 8)], [(1, 2), (3, 4), (7, 8)], [(1, 2), (7, 8)], [(1, 2), (3, 4), (5, 6), (7, 8)], [(1, 2)], tensor([], size=(0, 2)), [(1, 2), (7, 8)], [(1, 2), (3, 4), (5, 6), (7, 8)], [(1, 2), (3, 4), (5, 6), (7, 8)], [(1, 2), (3, 4), (7, 8)], [(1, 2), (3, 4), (5, 6), (7, 8)], [(1, 2), (3, 4), (5, 6), (7, 8)], [(1, 2), (3, 4)]]


In [50]:
bin_seq_lens = torch.tensor([len(x) for x in new_unfold_spikes]) 
it_len = iter(bin_seq_lens)
bin_seq_lens_restored = [torch.tensor(list(islice(it_len, l))) for l in bins_per_trial]
padded_seq_lens = pad_sequence(bin_seq_lens_restored, batch_first=True, padding_value=0)
print(padded_seq_lens)

tensor([[4, 3, 2, 4],
        [1, 0, 2, 4],
        [4, 3, 4, 0],
        [4, 2, 0, 0]])


In [51]:
tensor_unfold_spikes = [torch.tensor(x) for x in new_unfold_spikes]
pad_unfold_data = pad_sequence(tensor_unfold_spikes, batch_first=True, padding_value=0)

C:\Users\17927\AppData\Local\Temp\ipykernel_20008\2963433758.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor_unfold_spikes = [torch.tensor(x) for x in new_unfold_spikes]


In [52]:
it = iter(pad_unfold_data)
padded_spikes = [list(islice(it, trial_len)) for trial_len in bins_per_trial]

In [53]:
torch_padded_spikes = [torch.stack(x) for x in padded_spikes]


In [54]:
padded_bin = pad_sequence(torch_padded_spikes, batch_first=True, padding_value=0)

In [55]:
print(padded_bin.shape)

torch.Size([4, 4, 4, 2])


In [56]:
trial_counts = torch.tensor(bins_per_trial)
max_bins = padded_bin.size(1) # 4
print(trial_counts)
print(max_bins)

tensor([4, 4, 3, 2])
4


In [57]:
bin_mask = torch.arange(max_bins)[None, :] < trial_counts[:, None]
print(bin_mask)

tensor([[ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True, False],
        [ True,  True, False, False]])


In [58]:
max_seq_len = padded_bin.size(2)
seq_range = torch.arange(max_seq_len).view(1, 1, -1)
len_target = padded_seq_lens.unsqueeze(-1)
final_mask = seq_range < len_target
print(final_mask)

tensor([[[ True,  True,  True,  True],
         [ True,  True,  True, False],
         [ True,  True, False, False],
         [ True,  True,  True,  True]],

        [[ True, False, False, False],
         [False, False, False, False],
         [ True,  True, False, False],
         [ True,  True,  True,  True]],

        [[ True,  True,  True,  True],
         [ True,  True,  True, False],
         [ True,  True,  True,  True],
         [False, False, False, False]],

        [[ True,  True,  True,  True],
         [ True,  True, False, False],
         [False, False, False, False],
         [False, False, False, False]]])
